In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# cwd = os.getcwd()
cwd = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
dir_name = "simulation"
mu_coeff = 3 # int(dir_name[1])
eta_coeff = 0 # int(dir_name[3])


In [ ]:
def list_matching_files_with_numbers(dir_name):
    matching_files = []
    dpy_pattern = re.compile(r"^dpy\d+$")  # Matches "dpy<int>"
    salida_pattern1 = re.compile(r"^salida(\d+)\.out$")  # Captures number after "salida"
    salida_pattern2 = re.compile(r"^salida(\d+)\_undone.out$")  # Captures number after "salida"

    if not os.path.isdir(dir_name):
        return []

    for dpy_dir in os.listdir(dir_name):
        dpy_path = os.path.join(dir_name, dpy_dir)
        if os.path.isdir(dpy_path) and dpy_pattern.match(dpy_dir):
            for file in os.listdir(dpy_path):
                match1 = salida_pattern1.match(file)
                match2 = salida_pattern2.match(file)
                match = match1 or match2
                if match:
                    salida_number = int(match.group(1))  # Extract the number
                    matching_files.append((os.path.join(dpy_path, file), salida_number))

    return matching_files

def read_simu(file):
    return pd.read_csv(file,
                       header=None,
                       sep='\\s+',
                       index_col=None,
                       names=["nsim", "t", "a", "e", "M", "w", "R", "mass", "dist", "spin", "ast_mass"],
                       usecols=range(11),
                      )

In [ ]:
files = list_matching_files_with_numbers(os.path.join(cwd, dir_name))

In [ ]:
simus = {}
if len(files) > 0:
    for file_num in files:
        simus[file_num[1]] = read_simu(file_num[0])
    nsimus = len(simus)
    print(f"{nsimus} simulations found in {dir_name}.")
elif os.path.isfile(os.path.join(cwd, dir_name, "salida.out")):
    salida = read_simu(os.path.join(cwd, dir_name, "salida.out"))
    isimus = salida["nsim"].unique()
    for isimu in isimus:
        simus[isimu] = salida[salida["nsim"] == isimu]
    nsimus = len(simus)
    del salida
    print("All simulations completed.")
else:
    raise FileNotFoundError(f"No matching files found in {dir_name} or salida.out file not found.")

In [ ]:
keys = list(simus.keys())

G = 4.9823394e-10  # [km^3 kg^(-1) day^(-2)]
mAst = simus[keys[1]].ast_mass.iloc[0]
spin = simus[keys[1]].spin.iloc[0]
GmAst = G * mAst

In [ ]:
def a_from_ratio(ratio):
    one_n2 = (ratio / spin)**2
    return (mu_coeff * one_n2)**(1/3.)

def ratio_from_a(a):
    one_n = np.sqrt(a**3 / mu_coeff)
    return spin * one_n

def days_from_spin(spin):
    return 2 * np.pi / spin

def spin_from_days(days):
    return 2 * np.pi / days

# Corotation radius and SOR
corot_radius = (GmAst/spin**2)**(1/3)  # km
def a_res(res, spin=spin):
    return res**(2/3) * (GmAst/spin**2)**(1/3)

# Paardekooper
e_f = lambda a, mu, R: 0.75 * mu / (1 + mu)**2 * (R / a)**2

# Angular momentum
e_am = lambda a, am: np.sqrt(1 - am / a)

In [ ]:
plt.figure(dpi=150)

for simu in simus.values():
    plt.plot(simu.t/365.25, simu.a, "-", lw=0.4)

plt.semilogx()

plt.xlabel("time [yrs]")
plt.ylabel("$a$ [km]")

plt.title("$\\mu=10^{-%d}$   $\\eta=10^{-%d}$"%(mu_coeff, eta_coeff))

plt.tick_params(axis="both", top=True, right=False, direction="inout", which="both")
plt.grid(alpha=0.4)

plt.show()

In [ ]:
plt.figure(dpi=150)

for simu in simus.values():
    plt.plot(simu.t/365.25, simu.R, "-", lw=0.4)

plt.semilogx()

plt.xlabel("time [yrs]")
plt.ylabel("$\\Omega/n$")

plt.title("$\\mu=10^{-%d}$   $\\eta=10^{-%d}$"%(mu_coeff, eta_coeff))

# plt.yticks(np.linspace(1.5, 4, 6))

# ax2 = plt.gca().secondary_yaxis("right", functions=(a_from_ratio, ratio_from_a))
# ax2.set_ylabel("$a$ [km]")

plt.tick_params(axis="both", top=True, right=False, direction="inout", which="both")
plt.grid(alpha=0.4)

plt.show()

In [ ]:
plt.figure(dpi=150)

for simu in simus.values():
    plt.plot(simu.R, simu.e, "-", lw=0.4)

plt.xlabel("$R$")
plt.ylabel("$e$")

plt.title("$\\mu=10^{-%d}$   $\\eta=10^{-%d}$"%(mu_coeff, eta_coeff))

plt.tick_params(axis="both", top=True, right=False, direction="inout", which="both")
plt.grid(alpha=0.4)

plt.show()

In [ ]:
plt.figure(dpi=220, figsize=(5,3))

for simu in simus.values():
    plt.plot(simu.t/365.25, simu.e, "-", lw=0.4)

plt.xlabel("$t$ [yrs]")
plt.ylabel("$e$")

plt.title("$\\mu=10^{-%d}$   $\\eta=10^{-%d}$"%(mu_coeff, eta_coeff))

plt.semilogx()
plt.semilogy()

plt.tick_params(axis="both", top=True, right=True, direction="inout", which="both")
plt.grid(alpha=0.4)

plt.show()

In [ ]:
plt.figure(dpi=220, figsize=(5,3))

ss = [2, 5]

for i, s in enumerate(ss):
    color = f"C{i}"
    plt.plot(simus[s].t/365.25, simus[s].e,
             ".--",
             lw=0.4,
             ms=0.2,
             color=color,
             label=f"$R_{s}$: {simus[s].R.iloc[0]:.2f}, $e_{s}$: {simus[s].e.iloc[0]:.0e}",
             )
    plt.axhline(y=simus[s].e.max(), lw=1, ls="--", color=color)
    plt.axhline(y=simus[s].e.min(), lw=1, ls="--", color=color)
    plt.axhline(y=simus[s].e.iloc[0], lw=1, ls="-", color=color)
    # plt.plot(simus[s].t/365.25, e_f(simus[s].a, 10**(-mu_coeff), 129), "-", lw=2, color="k")


plt.xlabel("$t$ [yrs]")
plt.ylabel("$e$")

plt.semilogx()
plt.semilogy()

plt.tick_params(axis="both", top=True, right=True, direction="inout", which="both")

plt.legend(loc="center", bbox_to_anchor=(0.5,1.1), ncols=2, edgecolor="none")

plt.grid(alpha=0.4)

plt.show()